## 1. Import Libraries

In [1]:
!pip install xgboost

In [2]:
!pip install feature-engine

In [3]:
!pip install optuna

In [28]:
import os
import warnings

import pickle

import numpy as np

import pandas as pd

import sklearn

from xgboost import XGBRegressor

from sklearn.metrics import r2_score

from sklearn.base import BaseEstimator,TransformerMixin

from sklearn.metrics.pairwise import rbf_kernel

from sklearn.pipeline import (
        Pipeline,
        FeatureUnion
)

from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import (
        OneHotEncoder,
        MinMaxScaler,
        PowerTransformer,
        FunctionTransformer,
        StandardScaler,
        OrdinalEncoder
)

from feature_engine.encoding import (
        RareLabelEncoder,
        MeanEncoder,
        CountFrequencyEncoder
)

from feature_engine.datetime import DatetimeFeatures

from feature_engine.outliers import Winsorizer

from feature_engine.selection import SelectBySingleFeaturePerformance

import optuna

## 2. Display Settings

In [29]:
pd.options.display.max_columns=None

In [30]:
sklearn.set_config(transform_output='pandas')

In [31]:
warnings.filterwarnings('ignore')

## 3. Read Datasets

In [32]:
train = pd.read_csv('train.csv')
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-Flight Meal Not Included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-Flight Meal Not Included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-Flight Meal Not Included,11087
...,...,...,...,...,...,...,...,...,...,...
6690,Jet Airways,2019-03-21,Delhi,Cochin,10:45:00,18:50:00,1925,2.0,No Info,11093
6691,Air India,2019-05-01,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8891
6692,Jet Airways,2019-06-01,Delhi,Cochin,14:00:00,19:00:00,300,1.0,In-Flight Meal Not Included,10262
6693,Air Asia,2019-06-24,Delhi,Cochin,07:55:00,13:25:00,330,1.0,No Info,6152


In [34]:
val = pd.read_csv('validation.csv')
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-Flight Meal Not Included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704
...,...,...,...,...,...,...,...,...,...,...
1669,Spicejet,2019-05-01,Chennai,Kolkata,09:45:00,12:00:00,135,0.0,No Info,3597
1670,Indigo,2019-05-01,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No Info,5069
1671,Jet Airways,2019-05-27,Delhi,Cochin,05:30:00,12:35:00,425,2.0,In-Flight Meal Not Included,15544
1672,Jet Airways,2019-06-12,Mumbai,Hyderabad,19:35:00,21:05:00,90,0.0,In-Flight Meal Not Included,3210


In [35]:
test = pd.read_csv('test.csv')
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
...,...,...,...,...,...,...,...,...,...,...
2088,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,12:35:00,1040,1.0,In-Flight Meal Not Included,12898
2089,Multiple Carriers,2019-06-27,Delhi,Cochin,11:25:00,19:15:00,470,1.0,No Info,7155
2090,Jet Airways,2019-06-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,In-Flight Meal Not Included,11627
2091,Multiple Carriers,2019-06-06,Delhi,Cochin,15:15:00,01:30:00,615,1.0,No Info,6795


## 4. Preprocessing Operations

In [36]:
# airline
airline_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('grouper',RareLabelEncoder(tol=0.1,replace_with='other',n_categories=2)),
    ('encoder',OneHotEncoder(sparse_output=False,handle_unknown='ignore'))
])

# doj
feature_to_extract = ['month','week','day_of_week','day_of_year']

doj_transformer = Pipeline(steps=[
    ('dt',DatetimeFeatures(features_to_extract=feature_to_extract,yearfirst=True,format='mixed')),
    ('scaler',MinMaxScaler())
])

# source & destination
location_pipe = Pipeline(steps=[
                         ('grouper',RareLabelEncoder(tol=0.1,replace_with='other',n_categories=2)),
                         ('encoder',MeanEncoder()),
                         ('scaler',PowerTransformer())        
])


def is_north(X):
    columns = X.columns.to_list()
    north_cities = ['Delhi','Kolkata','Mumbai','New Delhi']
    return (
        X
        .assign(**{
            f"{col}_is_north" : X.loc[:,col].isin(north_cities).astype('int')
            for col in columns
        })
        .drop(columns=columns)
    )


location_transformer = FeatureUnion(transformer_list=[
    ('pipe1',location_pipe),
    ('pipe2',FunctionTransformer(func=is_north))
])


# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ('dt',DatetimeFeatures(features_to_extract=['hour','minute'])),
    ('scaler',MinMaxScaler())
])


def part_of_day(X,morning=4,afternoon=12,evening=16,night=20):
    columns = X.columns.to_list()
    
    X_temp = X.assign(**{
        col : pd.to_datetime(X.loc[:,col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day":np.select(
                [X_temp.loc[:,col].between(morning,afternoon,inclusive='left'),
                X_temp.loc[:,col].between(afternoon,evening,inclusive='left'),
                X_temp.loc[:,col].between(evening,night,inclusive='left')],
                ['morning','afternoon','evening'],
                default='night'
            )
            for col in columns
        })
        .drop(columns=columns)
    )


time_pipe2 = Pipeline(steps=[
    ('part',FunctionTransformer(func=part_of_day)),
    ('encoder',CountFrequencyEncoder()),
    ('scaler',MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ('pipe1',time_pipe1),
    ('pipe2',time_pipe2)
])


# duration
class RBFPercentileSimilarity(BaseEstimator,TransformerMixin):
   
    def __init__(self,variables=None,percentile=[0.25,0.5,0.75],gamma=0.1):
        self.variables=variables
        self.percentile=percentile
        self.gamma=gamma

    def fit(self,X,y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include='number').columns.to_list()

        self.reference_values = {
            col : (
                X
                .loc[:,col]
                .quantile(self.percentile)
                .values
                .reshape(-1,1)
            )
            for col in self.variables
        }

        return self



    def transform(self,X):
        objects = []

        # create column name.
        for col in self.variables:
            
            columns = [f"{col}_rbf_{int(percentile*100)}" for percentile in self.percentile]

            obj = pd.DataFrame(
                data = rbf_kernel(X.loc[:,[col]],Y=self.reference_values[col],gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)

        return pd.concat(objects,axis=1)
    
    
def duration_category(X,short=180,med=420):
    return (
        X
        .assign(
            duration_cat = np.select([(X.duration.lt(short)),
                                      (X.duration.between(short,med,inclusive='left'))],
                                     ['short','medium'],
                                     default='long')
        )
        .drop(columns = 'duration')
    )


def is_over(X,value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}" : X.duration.ge(value).astype('int')
        })
        .drop(columns='duration')
    )


duration_pipe1 = Pipeline(steps=[
    ('rbf',RBFPercentileSimilarity()),
    ('scaler',PowerTransformer())
])


duration_pipe2 = Pipeline(steps=[
    ('category',FunctionTransformer(func=duration_category)),
    ('encoder',OrdinalEncoder(categories=[['short','medium','long']]))
])


duration_union = FeatureUnion(transformer_list=[
    ('part1',duration_pipe1),
    ('part2',duration_pipe2),
    ('part3',FunctionTransformer(func=is_over)),
    ('part4',StandardScaler())
])


duration_transformer = Pipeline(steps=[
    ('outliers',Winsorizer(capping_method='iqr',fold=1.5)),
    ('imputer',SimpleImputer(strategy='median')),
    ('union',duration_union)
])



# total_stops
def is_direct(X):

    return (

        X
        .assign(
                is_direct_fight = X.total_stops.eq(0).astype(int)
        )
    )

total_stops_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('',FunctionTransformer(func=is_direct))
])

# additional_info
add_info_pipe1 = Pipeline(steps=[
                ('grp',RareLabelEncoder(tol=0.1,n_categories=2,replace_with='other')),
                ('encoder',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])


def have_info(X):
    return (
        X
        .assign(additional_info = X.additional_info.ne('No Info').astype(int))
    )

info_union = FeatureUnion(transformer_list=[
    ('part1',add_info_pipe1),
    ('part2',FunctionTransformer(func=have_info))
])


add_info_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',fill_value='unknown')),
    ('union',info_union)
])



# column_transformer
col_transform = ColumnTransformer(transformers=[
    ('air',airline_transformer,['airline']),
    ('doj',doj_transformer,['date_of_journey']),
    ('location',location_transformer,['source','destination']),
    ('time',time_transformer,['dep_time','arrival_time']),
    ('duration',duration_transformer,['duration']),
    ('stops',total_stops_transformer,['total_stops']),
    ('info',add_info_transformer,['additional_info'])
],remainder='passthrough')


# feature selector
estimator = RandomForestRegressor(n_estimators=10,max_depth=3,random_state=42)


selector = SelectBySingleFeaturePerformance(
    estimator=estimator,
    scoring='r2',
    threshold=0.1
)


# preprocessor
preprocessor = Pipeline(steps=[
    ('transformer',col_transform),
    ('selector',selector)
])

In [37]:
preprocessor.fit(
    train.drop('price',axis=1),
    train.price
)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt'...
                                                                                                                                     replace_with='other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x00000171D161C2C0>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

## 5. Apply Preprocessing Operations

In [38]:
preprocessor.transform(train.drop('price',axis=1))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,duration__duration_rbf_25,duration__duration_cat,duration__duration_over_1000,duration__duration,stops__total_stops,stops__is_direct_fight
0,0.0,1.0,0.0,0.176471,0.169492,-0.857930,-0.736484,-0.364262,2.0,0,-0.033916,1.0,0
1,0.0,1.0,0.0,0.235294,0.220339,1.065418,1.061694,-0.364262,2.0,0,0.046422,1.0,0
2,0.0,0.0,1.0,0.058824,0.067797,-0.857930,-0.736484,2.373008,0.0,0,-0.917631,0.0,1
3,0.0,0.0,0.0,0.882353,0.872881,-0.203928,-0.224351,-0.364262,2.0,0,-0.174507,1.0,0
4,0.0,1.0,0.0,0.117647,0.093220,-0.857930,-0.736484,-0.364262,2.0,0,-0.214676,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6690,0.0,1.0,0.0,0.176471,0.169492,1.065418,1.061694,-0.364262,2.0,1,2.597145,2.0,0
6691,0.0,0.0,0.0,0.529412,0.516949,-0.203928,-0.224351,-0.364262,2.0,0,-0.174507,1.0,0
6692,0.0,1.0,0.0,0.764706,0.779661,1.065418,1.061694,-0.364262,1.0,0,-0.666576,1.0,0
6693,0.0,0.0,1.0,1.000000,0.974576,1.065418,1.061694,-0.364262,1.0,0,-0.606322,1.0,0


In [39]:
X_train = preprocessor.transform(train.drop('price',axis=1))
y_train = train.price

In [41]:
X_val = preprocessor.transform(val.drop('price',axis=1))
y_val = val.price

In [42]:
X_test = preprocessor.transform(test.drop('price',axis=1))
y_test = test.price

## 6. Hyperparameter Tuning Using Optuna

In [17]:
from sklearn.metrics import r2_score

def objective(trial):
        param = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }
    
        model = XGBRegressor(**param, random_state=42)
        model.fit(X_train, y_train)
    
        preds = model.predict(X_val)
        r2 = r2_score(y_val, preds)
    
        return r2

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-06-23 02:13:30,145] A new study created in memory with name: no-name-ae033d1a-3679-415a-81a5-6e986f4f4a1c
[I 2024-06-23 02:13:30,332] Trial 0 finished with value: 0.7406265139579773 and parameters: {'n_estimators': 128, 'learning_rate': 0.1369547249913418, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8890277718860737, 'colsample_bytree': 0.5314541373559558, 'gamma': 1.9251455789803256, 'reg_alpha': 0.36120410535220837, 'reg_lambda': 0.3011737636979759}. Best is trial 0 with value: 0.7406265139579773.
[I 2024-06-23 02:13:31,056] Trial 1 finished with value: 0.6663304567337036 and parameters: {'n_estimators': 207, 'learning_rate': 0.2983514963095258, 'max_depth': 9, 'min_child_weight': 3, 'subsample': 0.8368477881963977, 'colsample_bytree': 0.6084044702247746, 'gamma': 4.389420712920808, 'reg_alpha': 0.12250260267834423, 'reg_lambda': 0.6625404896975932}. Best is trial 0 with value: 0.7406265139579773.
[I 2024-06-23 02:13:31,377] Trial 2 finished with value: 0.7290841341

In [19]:
best_params = study.best_params
best_params

{'n_estimators': 113,
 'learning_rate': 0.052784954059215884,
 'max_depth': 9,
 'min_child_weight': 10,
 'subsample': 0.6683809839193526,
 'colsample_bytree': 0.7543789330743793,
 'gamma': 0.21918736456489668,
 'reg_alpha': 0.8968270674035634,
 'reg_lambda': 0.8724367609097978}

## 7. Train the Model Using Best Parameters

In [20]:
final_model = XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7543789330743793, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.21918736456489668,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.052784954059215884,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=113, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [21]:
with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)

## 8. Model Evaluation

In [65]:
with open("xgb_model.pkl", "rb") as f:
    best_model = pickle.load(f)

In [66]:
r2_score(y_train,best_model.predict(X_train))

0.8421941995620728

In [67]:
r2_score(y_val,best_model.predict(X_val))

0.7718122005462646

In [68]:
r2_score(y_test,best_model.predict(X_test))

0.7687056660652161